In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from mongo import attraction_db
from model.google import Google
from model.Word import Word
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import random
import os
import json
import time
import csv
import os

c:\work\Google_Maps_Crawler\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# test db
existing_names = attraction_db.attractionInfo.distinct("name")
existing_names

In [81]:
url = 'https://www.google.com/maps/search/%E6%99%AF%E9%BB%9E'

driver = webdriver.Chrome()
driver.get(url)

In [3]:
def getPage():
    page_content = driver.page_source
    # 將 HTML 內容轉換成 BeautifulSoup 物件，html.parser 為使用的解析器
    soup = BeautifulSoup(page_content, 'html.parser')
    # 透過 select 使用 CSS 選擇器 選取我們要選的 html 內容
    return soup.select('.Nv2PK.THOPZb.CpccDe')

def scrollPage(times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        counter += 1

def getElement():  # 抓取目前的所有留言
    spots_list = []
    scrollPage(5)
    elements = getPage()

    for element in elements:
        name = element.select('.qBF1Pd.fontHeadlineSmall')[0].text
        url = element.select('.hfpxzc')[0].get('href')
        dataset = {}
        dataset['name'] = name
        dataset['google_url'] = url

        spots_list.append(dataset)
    return spots_list

In [297]:
lastSpot = ''
while 7 == 7:
    print(lastSpot)
    spots_list = getElement()
    if (spots_list[len(spots_list)-1]["name"] == lastSpot ):
        break
    lastSpot = spots_list[len(spots_list)-1]["name"]
    time.sleep(1)
    getElement()

csv_file = './data/spot_list/spots.csv'
if os.path.exists(csv_file):
    with open(csv_file, mode="r", newline="", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        existing_data = list(reader)
else:
    existing_data = []

for spot in spots_list:
    existing_data.append(spot)

print(len(existing_data))


with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    # 定義 CSV 欄位
    fieldnames = ["name", "google_url"]
    # 建立 CSV 寫入器
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

    print(f"已將資料寫入 CSV 檔案：{csv_file}")


後浦16藝文特區
金水學校展示館
清遠湖
7611
已將資料寫入 CSV 檔案：./data/spot_list/spots.csv


In [300]:
dataset = pd.read_csv('./data/spot_list/spots.csv')
dataset

,name,google_url
0,淡水老街,https://www.google.com/maps/place/%E6%B7%A1%E6...
1,重建街戀愛巷,https://www.google.com/maps/place/%E9%87%8D%E5...
2,一滴水紀念館,https://www.google.com/maps/place/%E4%B8%80%E6...
3,淡水情人塔 Lover's Tower,https://www.google.com/maps/place/%E6%B7%A1%E6...
4,情人橋,https://www.google.com/maps/place/%E6%83%85%E4...
...,...,...
7606,官澳風獅爺(雄),https://www.google.com/maps/place/%E5%AE%98%E6...
7607,黃卓彬洋樓,https://www.google.com/maps/place/%E9%BB%83%E5...
7608,夏墅風獅爺,https://www.google.com/maps/place/%E5%A4%8F%E5...
7609,建功嶼,https://www.google.com/maps/place/%E5%BB%BA%E5...


In [310]:
df_unique = dataset.drop_duplicates(subset="name", keep="first")
df_unique.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")
df_unique

,name,google_url
0,淡水老街,https://www.google.com/maps/place/%E6%B7%A1%E6...
1,重建街戀愛巷,https://www.google.com/maps/place/%E9%87%8D%E5...
2,一滴水紀念館,https://www.google.com/maps/place/%E4%B8%80%E6...
3,淡水情人塔 Lover's Tower,https://www.google.com/maps/place/%E6%B7%A1%E6...
4,情人橋,https://www.google.com/maps/place/%E6%83%85%E4...
...,...,...
7606,官澳風獅爺(雄),https://www.google.com/maps/place/%E5%AE%98%E6...
7607,黃卓彬洋樓,https://www.google.com/maps/place/%E9%BB%83%E5...
7608,夏墅風獅爺,https://www.google.com/maps/place/%E5%A4%8F%E5...
7609,建功嶼,https://www.google.com/maps/place/%E5%BB%BA%E5...


In [312]:
existing_names = attraction_db.attractionInfo.distinct("name")
df_filtered = df_unique[~df_unique["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")

In [24]:
existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/random_spotsList.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList_clean.csv', index=False, encoding="utf-8-sig")
df_filtered

,name,google_url
1,精英溫泉,https://www.google.com/maps/place/%E7%B2%BE%E8...
27,龍鳳海灘,https://www.google.com/maps/place/%E9%BE%8D%E9...
50,七星潭海岸風景特定區,https://www.google.com/maps/place/%E4%B8%83%E6...
67,謙桂緣,https://www.google.com/maps/place/%E8%AC%99%E6...
86,林口．樂活公園/捷運主題公園,https://www.google.com/maps/place/%E6%9E%97%E5...
...,...,...
2396,錫安山,https://www.google.com/maps/place/%E9%8C%AB%E5...
2397,朴子藝術公園,https://www.google.com/maps/place/%E6%9C%B4%E5...
2398,埤頭木棉花道,https://www.google.com/maps/place/%E5%9F%A4%E9...
2399,福壽山農場｜千櫻園,https://www.google.com/maps/place/%E7%A6%8F%E5...


In [3]:
# ============================================
# TEST
# ============================================

spotsList = pd.read_csv('./data/spot_list/spotsList_clean.csv') 

for index, row in spotsList[105:120].iterrows():
    print(index,row['name'],row['google_url'])

105 巴克禮紀念公園 https://www.google.com/maps/place/%E5%B7%B4%E5%85%8B%E7%A6%AE%E7%B4%80%E5%BF%B5%E5%85%AC%E5%9C%92/data=!4m7!3m6!1s0x346e7421350b0a33:0x2dbbd32dfa02333c!8m2!3d22.9719929!4d120.2225874!16s%2Fg%2F155sx61t!19sChIJMwoLNSF0bjQRPDMC-i3Tuy0?authuser=0&hl=zh-TW&rclk=1
106 竹山天梯 https://www.google.com/maps/place/%E7%AB%B9%E5%B1%B1%E5%A4%A9%E6%A2%AF/data=!4m7!3m6!1s0x346ed038123d013f:0xe4a5e4d318b7164a!8m2!3d23.669172!4d120.734196!16s%2Fg%2F11_ylslbc!19sChIJPwE9EjjQbjQRSha3GNPkpeQ?authuser=0&hl=zh-TW&rclk=1
107 李澤藩美術館 https://www.google.com/maps/place/%E6%9D%8E%E6%BE%A4%E8%97%A9%E7%BE%8E%E8%A1%93%E9%A4%A8/data=!4m7!3m6!1s0x346835ea5a54aad1:0x995bed83bdf81799!8m2!3d24.8013329!4d120.9680739!16s%2Fg%2F11fd6dglc1!19sChIJ0apUWuo1aDQRmRf4vYPtW5k?authuser=0&hl=zh-TW&rclk=1
108 天送埤林鐵文創園區（太平山林鐵） https://www.google.com/maps/place/%E5%A4%A9%E9%80%81%E5%9F%A4%E6%9E%97%E9%90%B5%E6%96%87%E5%89%B5%E5%9C%92%E5%8D%80%EF%BC%88%E5%A4%AA%E5%B9%B3%E5%B1%B1%E6%9E%97%E9%90%B5%EF%BC%89/data=!4m7!3m6!1s0x34680

In [4]:
# 取得 Google URL

name_to_url = {}
with open('./data/spot_list/spotsList.csv', 'r', encoding='utf-8') as spots_file:
    spots_reader = csv.reader(spots_file)
    next(spots_reader)  # 跳过标题行
    for row in spots_reader:
        name = row[0]  # 假设名称在第一列
        url = row[1]   # 假设URL在第二列
        name_to_url[name] = url

def get_url_for_name(name):
    return name_to_url.get(name, "URL not found")

In [5]:
# ============================================
# 個別處理
# ============================================

name = '講美鎮海社區海堤公園'
url  = get_url_for_name(name)

In [24]:
data = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
tag_value = data['words'].tolist()

place_detail_data = Google().get_place_info(name,url,tag_value)
place_detail_data = json.loads(place_detail_data)
print(place_detail_data)

existing_data = attraction_db.attractionInfo.find_one({
    "$or": [
        {"place_id": place_detail_data["place_id"]},
        {"id": place_detail_data["id"]}
    ]
})
print(existing_data)
if existing_data is None:
    attraction_db.attractionInfo.insert_one(place_detail_data)
    print(f"{place_detail_data['name']}：已新增")

{'name': '櫻花大道', 'google_url': 'https://www.google.com/maps/place/%E6%AB%BB%E8%8A%B1%E5%A4%A7%E9%81%93/data=!4m7!3m6!1s0x3442ae39be7123e3:0x76aeeeafb2dd3e9e!8m2!3d25.1504227!4d121.5044983!16s%2Fg%2F11b720r0xy!19sChIJ4yNxvjmuQjQRnj7dsq_urnY?authuser=0&hl=zh-TW&rclk=1', 'address': '251新北市淡水區', 'id': 'TPH_271', 'region': '新北市', 'place_id': 'ChIJAQAAABCwQjQRLbiMeGSYq48', 'google_address': '251, Taiwan, New Taipei City, Tamsui District', 'rating': 4.1, 'user_ratings_total': 602, 'lat': 25.1687611, 'lng': 121.4614806, 'photo_reference': 'AUacShhuBu-96N4eyT7AUvHd0UnpMeSsenDHbGURoDY31itmMIPoJtP9FmzE9W3pJ8qJ2EGeXF1Hsfiz2iQUHKNhHcrv9lE3ZJPAzm-p4FueibFIVjiX_xBFURJ9A_HKHZ3zCjTDq6X6HdFvk2gNGlQnbM73skls_aG7BE9udbXaucOtX-NV', 'photo_url': 'https://lh3.googleusercontent.com/places/ANJU3DsqA6Zsa-w_iOVmBmTRj4mqfePGokgc8kUzGl_k_47ZMRqyJ9K6AJk6CN5oUOtp7Z48M9uHic_l84bYzkeFZFJLNW7n9yynNrg=s1600-w1200-h1200', 'tag': ['櫻花', '櫻花季', 'ㄧ些', '健走', '登山', '北投區', '復興三路', '櫻花樹']}
{'_id': ObjectId('64eb5a10bcda7bf0f797

In [10]:
# 檢查未成功插入資料庫的景點

existing_names = set(doc['name'] for doc in attraction_db.attractionInfo.find({}, {"name": 1}))

csv_folder_path = './data/tfidf_csv/' 
csv_files = os.listdir(csv_folder_path)
new_csv_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
print(len(new_csv_files))
print(new_csv_files)

0
[]


In [11]:
# 刪除不必要的景點
for spot in new_csv_files:
    name = spot
    csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"{name}.csv 已删除")

國家級照相館淡金路分館.csv 已删除
山崎尾.csv 已删除
山崗石材.csv 已删除
康荷賞花園.csv 已删除
懷恩公園牌樓.csv 已删除
成德公園.csv 已删除
本山地質公園.csv 已删除
柳杉造林地.csv 已删除
淡水區樹林口溪十一橋.csv 已删除
淡水新埔橋.csv 已删除
過港隧道.csv 已删除
酋長岩.csv 已删除
金龍湖.csv 已删除
阿桂俱樂部.csv 已删除
鼻頭角公園.csv 已删除


In [9]:
# 新增缺少的景點
for spot in new_csv_files:
    name = spot
    url = get_url_for_name(name)
    try:

        tfidf = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
        tag_value = tfidf['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
            csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
            if os.path.exists(csv_file_path):
                os.remove(csv_file_path)
                print(f"{place_detail_data['name']}.csv 已删除")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue

林口．樂活公園｜捷運主題公園：未新增
林口．樂活公園｜捷運主題公園.csv 已删除


In [8]:
# 隨機排序景點

list = pd.read_csv('./data/spot_list/spotsList.csv')
print(list)

random.seed(42)
random_spotsList = list.sample(frac=1,random_state=random.randint(1,10000))
print(random_spotsList)

random_spotsList.to_csv('./data/spot_list/random_spotsList.csv',encoding="utf-8-sig",index=False)


        name                                         google_url
0      嶺腳觀景台  https://www.google.com/maps/place/%E5%B6%BA%E8...
1       嶺腳瀑布  https://www.google.com/maps/place/%E5%B6%BA%E8...
2       平溪瀑布  https://www.google.com/maps/place/%E5%B9%B3%E6...
3       平溪老街  https://www.google.com/maps/place/%E5%B9%B3%E6...
4       菁桐老街  https://www.google.com/maps/place/%E8%8F%81%E6...
...      ...                                                ...
2396  八二三戰史館  https://www.google.com/maps/place/%E5%85%AB%E4...
2397   田浦風獅爺  https://www.google.com/maps/place/%E7%94%B0%E6...
2398   黃卓彬洋樓  https://www.google.com/maps/place/%E9%BB%83%E5...
2399     建功嶼  https://www.google.com/maps/chplace/%E5%BB%BA%...
2400     清遠湖  https://www.google.com/maps/place/%E6%B8%85%E9...

[2401 rows x 2 columns]
               name                                         google_url
2264     將軍區長榮里木棉花道  https://www.google.com/maps/place/%E5%B0%87%E8...
1536           精英溫泉  https://www.google.com/maps/place/%E7%B2%BE%

In [21]:
# ============================================
# 個別處理 2
# ============================================
ws_driver = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")

dataset = pd.read_excel('./data/set/三貂嶺生態友善隧道.xlsx')
tag = Word(ws_driver,pos_driver).save(dataset)
tag['words'].tolist()

['隧道',
 '蝙蝠',
 '三貂嶺',
 '步道',
 '涼爽',
 '生態',
 '牡丹',
 '自行車',
 '自行車道',
 '腳踏車',
 '體驗',
 '鋼筋',
 'ubike',
 '散步',
 '騎車',
 '自然',
 '拍照',
 '天然',
 '導覽',
 '單車',
 '牡丹站',
 '歷史',
 '隧道口',
 '鏡面',
 '猴硐',
 '悠閒',
 '健行',
 '鐘乳石',
 '水池',
 '消暑',
 '老街',
 '懷舊',
 '清涼',
 '避暑',
 '鐵條',
 '鐵路',
 '文化',
 '瑞芳',
 '牡丹端',
 '工法',
 '法國',
 '侯硐',
 '生態環境',
 '三貂嶺站',
 '休憩站',
 '鐵道',
 '總長',
 '時空',
 '區間車',
 '雙溪',
 '台鐵',
 '企鵝妹',
 '鐵軌',
 '古蹟',
 '健走',
 '大自然',
 '新北市',
 '報到處',
 '設計師',
 '電輔車',
 '普悠瑪',
 '祕境',
 '金字橋',
 '步道行',
 '人車',
 '網站',
 '古跡',
 '曬太陽',
 '鋪面',
 '植物',
 '森林',
 '設計感',
 '青山',
 '新北',
 '綠意盎然',
 '出口處',
 '畫布',
 '清幽',
 '里程',
 '偏鄉',
 '氣温',
 '噪音',
 '區公所',
 '路道',
 '高跟鞋',
 '老少咸宜',
 '水滴',
 '原住民',
 '秀麗',
 '蝙幅',
 '終點站',
 '場站',
 '周休',
 '草嶺',
 '制度',
 '博物館',
 '縫隙',
 '煤灰',
 '首獎',
 '葉鼻蝠',
 '總額',
 '地質',
 '清新',
 '朝聖',
 '腳程',
 '路基',
 '歴史',
 '瀑布群',
 '愜意',
 '動物',
 '蛙蝦',
 '牡丹街',
 '下雨聲',
 '閃光',
 '山蝦',
 '毛蟹魚',
 '內涵',
 '猴硐站',
 '解說人',
 '基隆河',
 '貴婦',
 '所有人',
 '長度',
 '郊遊',
 '清靜',
 '幽靜',
 '空隙',
 '頭燈',
 '野生',
 '焦點',
 '棧道',
 '猴桐',
 '山景',
 '總量',
 '雨傘',
 '三雕嶺站'

In [22]:
csv_folder_path = './data/set/' 
csv_files = os.listdir(csv_folder_path)
tfidf_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
tfidf_files

['三貂嶺生態友善隧道', '十分老街', '嘉義公園射日塔', '後壁湖', '雙溪荷花園']

In [23]:
for spot in tfidf_files:
    name = spot
    url = get_url_for_name(name)
    try:
        dataset = pd.read_excel('./data/set/{}.xlsx'.format(name))
        tag = Word(ws_driver,pos_driver).save(dataset)
        tag_value = tag['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue

三貂嶺生態友善隧道：已新增
十分老街：已新增
嘉義公園射日塔：已新增
後壁湖：已新增
雙溪荷花園：已新增
